<a href="https://colab.research.google.com/github/HagarIbrahiem/Learning_purposes/blob/main/Handle_Missing_Values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Connect to Google drive for Datasets

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
!ls drive/'My Drive'/ML_Process_Data_Files/Section_5_Data_Preprocessing

clv_data.csv


In [24]:
import random
import pandas as pd


df = pd.read_csv("drive/My Drive/ML_Process_Data_Files/Section_5_Data_Preprocessing/clv_data.csv")

df['lifetime_value'] = df['purchases'] * 20

df.head()

,Unnamed: 0,id,age,gender,income,days_on_platform,city,purchases,lifetime_value
0,0,0,NaN,Male,126895,14.0,San Francisco,0,0
1,1,1,NaN,Male,161474,14.0,Tokyo,0,0
2,2,2,24.0,Male,104723,34.0,London,1,20
3,3,3,29.0,Male,43791,28.0,London,2,40
4,4,4,18.0,Female,132181,26.0,London,2,40


## Nullable values imputaions


There is no thump rule to handle missing values in a particular manner, the method which gets a robust model with the best performance. One can use various methods on different features depending on how and what the data is about. Having domain knowledge about the dataset is important, which can give an insight into how to preprocess the data and handle missing values.


Fun containts percentage of null values per column

In [118]:
def nulls_summary_table(df):
    """
    Returns a summary table showing null value counts and percentage
    
    Parameters:
    df (DataFrame): Dataframe to check
    
    Returns:
    null_values (DataFrame)
    """
    null_values = pd.DataFrame(df.isnull().sum())
    null_values[1] = (null_values[0]/len(df) ) *100
    null_values.columns = ['null_count','null_pct']
    return null_values

nulls_summary_table(df)

,null_count,null_pct
Unnamed: 0,0,0.00
id,0,0.00
age,2446,48.92
gender,0,0.00
income,0,0.00
days_on_platform,141,2.82
city,0,0.00
purchases,0,0.00
lifetime_value,0,0.00


**FIRST :Drop Null Values**

Dffierence between delete row and column

In [14]:
df2 = df.copy()
print(df2.isnull().sum().sum())



# drop entire column if it has so many null values 
df_deleterow = df2.drop( ['age'] , axis=1)                               ### see difference between dDROPNA row , DROP column
print(df_deleterow.isnull().sum().sum())



# drop entire row if it has so many null values 
df_deleterow = df2.dropna(axis=0)
print(df_deleterow.isnull().sum().sum())



2587
141


In [119]:
drop_df = df.copy()
drop_df = drop_df.dropna()


X_d = drop_df[['age','days_on_platform','income']]
y_d = drop_df['lifetime_value']


X_train_d = X_d[:4000]
y_train_d = y_d[:4000]

X_test_d = X_d[1000:]
y_test_d = y_d[1000:]

**SECOND: replacing with an arbbitary value**

in case you have an educated guess about the missing value 

In [15]:
arb_df = df.copy()
print (arb_df.isnull().sum().sum())
arb_df['age'] = arb_df['age'].fillna(50)
print (arb_df.isnull().sum().sum())

2587
141


**THIRD: Mean/Median/Mode Imputation**

The next is mean/median/mode imputation. We can use the native numpy functions for the mean and median. We can use scipy for the mode. Then, pandas as a native fillna method we can use to impute the nulls with the mean/median/mode:

there are tow ways to impute with Mean/Median/Mode 
1- Pandas + Numpy
2- Sklearn by using SimpleImputer Univaraiate Approcach 

In [35]:
m_df = df.copy()

X_m = m_df[['age','days_on_platform','income']]
y_m = m_df['lifetime_value']


X_train_m = X_m[:4000]
y_train_m = y_m[:4000]

X_test_m = X_m[1000:]
y_test_m = y_m[1000:]

using Pandas + Numby

In [36]:
import numpy as np
from scipy import stats

## Mean
X_train_m.loc[:,'age'] = X_train_m['age'].fillna(np.mean(X_train_m['age']))
X_test_m.loc[:,'age'] = X_test_m['age'].fillna(np.mean(X_train_m['age'])) ## Cannot use training dataset to impute

X_train_m.loc[:,'days_on_platform'] = X_train_m['days_on_platform'].fillna(np.mean(X_train_m['days_on_platform']))
X_test_m.loc[:,'days_on_platform'] = X_test_m['days_on_platform'].fillna(np.mean(X_train_m['days_on_platform'])) ## Cannot use training dataset to impute

## Median
X_train_m.loc[:,'age'] = X_train_m['age'].fillna(np.median(X_train_m['age']))

## Mode
X_train_m.loc[:,'age'] = X_train_m['age'].fillna(stats.mode(X_train_m['age'])[0][0])

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


using sklearn - simple imputer

In [56]:
from sklearn.impute import SimpleImputer


df_simpleImp = df.copy ()

_SimpleImputer = SimpleImputer(missing_values=np.nan, strategy='mean' )
df_simpleImp.age = _SimpleImputer.fit_transform(df_simpleImp['age'].values.reshape(-1,1))[:,0]

df_simpleImp.isnull().sum()




Unnamed: 0            0
id                    0
age                   0
gender                0
income                0
days_on_platform    141
city                  0
purchases             0
lifetime_value        0
dtype: int64

**FORTH: backword fill (using next value) OR forward fill (using previous value) OR Interploation (average of previous and next)**

In [25]:
import numpy as np
my_df = pd.Series(range(10))
my_df[2:5] = np.nan
my_df

0    0.0
1    1.0
2    NaN
3    NaN
4    NaN
5    5.0
6    6.0
7    7.0
8    8.0
9    9.0
dtype: float64

In [26]:
my_df.fillna(method='ffill')


0    0.0
1    1.0
2    1.0
3    1.0
4    1.0
5    5.0
6    6.0
7    7.0
8    8.0
9    9.0
dtype: float64

In [27]:
my_df.fillna(method='bfill')


0    0.0
1    1.0
2    5.0
3    5.0
4    5.0
5    5.0
6    6.0
7    7.0
8    8.0
9    9.0
dtype: float64

**FIFTH : Interploations**

In [29]:
##1- Linear Interpolation defualt method is LINEAR 
# It estimates the unknown value in the same increasing order from previous values
my_df.interpolate() 

0    0.0
1    1.0
2    2.0
3    3.0
4    4.0
5    5.0
6    6.0
7    7.0
8    8.0
9    9.0
dtype: float64

In [31]:
# 2- Polynomial Interpolation
#In Polynomial Interpolation you need to specify an order. 
#It means that polynomial interpolation is filling missing values with the lowest possible degree that passes through available data points.
# The polynomial Interpolation curve is like the trigonometric sin curve or assumes it like a parabola shape. 
my_df.interpolate(method="polynomial", order=3)

0    0.0
1    1.0
2    2.0
3    3.0
4    4.0
5    5.0
6    6.0
7    7.0
8    8.0
9    9.0
dtype: float64

In [32]:
## 3- Interpolation through Padding (act like Forwardfill)
# Interpolation with help of padding simply means filling missing values with the same value present above them in the dataset.
# If the missing value is in the first row then this method will not work.
# While using this technique you also need to specify the limit which means how many NaN values to fill.
my_df.interpolate(method="pad", limit=2)


0    0.0
1    1.0
2    1.0
3    1.0
4    NaN
5    5.0
6    6.0
7    7.0
8    8.0
9    9.0
dtype: float64

**SIXTH: More complicated imputation technique: Multiple Imputation / MultiVaraite Approach as it use multiple features to impute missing value**

1-Regression Imputation (linear and forests)

2- Nearest Neighbor Imputation


Multiple imputation has a few different estimators, using the `estimator` argument:

- `BayesianRidge`: Regularized Linear Regression

- `RandomForestRegressor`: Random Forest Model. Mimics missForest in the R language.

We'll go over how these estimators work in the next course: ML Algorithms. 

The `missing_values` argument is a placeholder for the data type of the missing values you want to impute. 

It's important to use `add_indicatorbool` as it'll create a placeholder indicating that you've imputed a missing value. This is important, because there could be patterns behind how a value is missing. Adding an indicator would allow you to keep track of where you made an imputation. Plus, it could also add signal into your model. 

`max_iter`: The number of iteration rounds.

In [58]:
from sklearn.linear_model import LinearRegression

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer

## Regression Imputation 

r_df = df.copy()

X_r = r_df[['age','days_on_platform','income']]
y_r = r_df['lifetime_value']


X_train_r = X_r[:4000]
y_train_r = y_r[:4000]

X_test_r = X_r[1000:]
y_test_r = y_r[1000:]

imp = IterativeImputer ( estimator=LinearRegression(), max_iter=10 , random_state= 0 
                        , add_indicator=True) #add_indicator add column to indicate if the value in imputed or not , in case null values are following patterns

# use 'days_on_platform','income' to predict 'age' , and use 'age' and 'income' to predict 'days_on_platform'
imp.fit(X_train_r)

X_train_r = imp.transform(X_train_r)
X_test_r = imp.transform (X_test_r)

In [59]:
X_train_r = pd.DataFrame(X_train_r)
X_train_r.columns = X_train_r.columns

X_test_r = pd.DataFrame(X_test_r)
X_test_r.columns = X_test_r.columns

df_r = pd.concat([X_train_r , X_test_r] , axis= 0)

In [60]:
X_train_r.isnull().sum()

0    0
1    0
2    0
3    0
4    0
dtype: int64

**Nearest Neighbor Imputation**

 you can also use nearest neighbors imputation. Nearest neighbor imputation essentially uses a K-Nearest Neighbors algorithm to find the most similar data points, to impute the null values.


`missing_values`: This is the type of null value you want to impute. Typically, this is NaN, but it could be float or whichever you decide.

`n_neighbors`: The number of neighbors to use for imputation. You can add more or less. Fewer neighbors can lead to overfitting. Larger numbers will lose some precision.

`weights`: Pick how you want to weight all points in each neighborhood. There are two typical ways: 'uniform' or 'distance'. Uniform is equal weighting. Distance is weighted by the distance from the point.

`callable` : a user-defined function which accepts an array of distances, and returns an array of the same shape containing the weights.
metric: The distance metric used to search for neighbors. The default is euclidean.

`add_indicator`: This will add a dummy feature 0 or 1 if the value was imputed, similar to add_indicatorbool.

In [62]:
knnImp = KNNImputer (weights='uniform' , n_neighbors=5) # for this null values , we will find the 5 closest values to get the impuation   

knnImp.fit(X_train_r)

X_train_k = knnImp.transform(X_train_r)
X_test_k = knnImp.transform(X_test_r)

y_train_k = y_train_r.copy()
y_test_k = y_test_r.copy()





array([[3.05480927e+01, 1.40000000e+01, 1.26895000e+05, 1.00000000e+00,
        0.00000000e+00],
       [3.08481372e+01, 1.40000000e+01, 1.61474000e+05, 1.00000000e+00,
        0.00000000e+00],
       [2.40000000e+01, 3.40000000e+01, 1.04723000e+05, 0.00000000e+00,
        0.00000000e+00],
       ...,
       [4.60000000e+01, 5.20000000e+01, 5.14320000e+04, 0.00000000e+00,
        0.00000000e+00],
       [1.40000000e+01, 3.70000000e+01, 4.94090000e+04, 0.00000000e+00,
        0.00000000e+00],
       [3.08868792e+01, 2.30000000e+01, 1.49963000e+05, 1.00000000e+00,
        0.00000000e+00]])

**use attripute of (add_indicator=True)**

In [63]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

X = pd.DataFrame({'Age':[20, 30, 10, np.nan, 10]})

imputer = SimpleImputer(add_indicator=True)
imputer.fit_transform(X)


# ‘1’ indicates that the corresponding value was missing and ‘0’ indicates that the corresponding value was not missing.

array([[20. ,  0. ],
       [30. ,  0. ],
       [10. ,  0. ],
       [17.5,  1. ],
       [10. ,  0. ]])

**SEVENTH : Imputation using Deep Learning Library — Datawig**

In [65]:
pip install datawig


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 KB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.6/29.6 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 86.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━

In [ ]:
import datawig

df_train, df_test = datawig.utils.random_split(data)

#Initialize a SimpleImputer model
imputer = datawig.SimpleImputer(
    input_columns=['days_on_platform','income'], # column(s) containing information about the column we want to impute
    output_column= 'age', # the column we'd like to impute values for
    output_path = 'imputer_model' # stores model data and metrics
    )

#Fit an imputer model on the train data
imputer.fit(train_df=df_train, num_epochs=50)

#Impute missing values and return original dataframe with predictions
imputed = imputer.predict(df_test)

**Compare between different Nullable Techniques**

In [126]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Drop Null Model
clf_n = RandomForestRegressor(random_state=0)
clf_n.fit(X_train_d, y_train_d)
pred_dropna = clf_n.predict(X_test_d)



# Mean Imputation Model
clf_m = RandomForestRegressor(random_state=0)
clf_m.fit(X_train_m, y_train_m)
pred_m = clf_m.predict(X_test_m)

# Regression Imputation
clf_r = RandomForestRegressor(random_state=0)
clf_r.fit(X_train_r, y_train_r)
pred_r = clf_r.predict(X_test_r)

#Nearest Neighbor Imputation
clf_n = RandomForestRegressor(random_state=0)
clf_n.fit(X_train_k, y_train_k)
pred_k = clf_n.predict(X_test_k)


print('Drop Null MAE Score: %.3f' % mean_absolute_error(y_test_d,pred_dropna))
print('Mean Impute MAE Score: %.3f' % mean_absolute_error(y_test_m,pred_m))
print('Regression MAE Score: %.3f '% mean_absolute_error(y_test_r,pred_r))
print('Nearest Neighbor MAE Score: %.3f'% mean_absolute_error(y_test_k,pred_k))

Drop Null MAE Score: 7.636
Mean Impute MAE Score: 10.828
Regression MAE Score: 10.796 
Nearest Neighbor MAE Score: 10.796


**Resources**


1  [link text](https://learn.365datascience.com/courses/learn-machine-learning-process-a-z/types-of-null-values/)


2  [link text](https://www.analyticsvidhya.com/blog/2021/10/handling-missing-value/#:~:text=Types%20Of%20Missing%20Values,Missing%20Not%20At%20Random%20(MNAR))